In [75]:
!pip install PyRogue

In [1]:
#Import generic libraries
import numpy as np
import pandas as pd
import torch

In [2]:
#loading the CNN dataset
news = pd.read_csv('/content/articles.csv')
DOCUMENT="Article Body"

#Because it is just a course we select a small portion of News.
MAX_NEWS = 3
subset_news = news.head(MAX_NEWS)

subset_news.head()

,Unnamed: 0,Published Date,Author,Source,Article Header,Sub_Headings,Article Body,Url
0,0,"July 7, 2023",Adam Zewe,MIT News Office,Learning the language of molecules to predict ...,This AI system only needs a small amount of da...,['Discovering new materials and drugs typicall...,https://news.mit.edu/2023/learning-language-mo...
1,1,"July 6, 2023",Alex Ouyang,Abdul Latif Jameel Clinic for Machine Learning...,MIT scientists build a system that can generat...,"BioAutoMATED, an open-source, automated machin...",['Is it possible to build machine-learning mod...,https://news.mit.edu/2023/bioautomated-open-so...
2,2,"June 30, 2023",Jennifer Michalowski,McGovern Institute for Brain Research,"When computer vision works more like a brain, ...",Training artificial neural networks with data ...,"['From cameras to self-driving cars, many of t...",https://news.mit.edu/2023/when-computer-vision...


In [27]:
articles = subset_news[DOCUMENT].tolist()


In [3]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#The model names are stored in the variables model_name_small and model_name_reference.
model_name_small = "t5-base"
model_name_reference = "flax-community/t5-base-cnn-dm"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.0 MB/s eta 0:00:00


In [5]:
#This function returns the tokenizer and the Model.
def get_model(model_id):
 tokenizer = AutoTokenizer.from_pretrained(model_id)
 model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

 return tokenizer, model

In [6]:
#Using the get_model function, we can obtain the two tokenizers and models:
tokenizer_small, model_small = get_model(model_name_small)
tokenizer_reference, model_reference = get_model(model_name_reference)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
# To generate the summaries, we will create a function that takes four parameters:

# A list of texts to summarize.
# The tokenizer.
# The model.
# The maximum length for the obtained summary.

In [29]:
def create_summaries(texts_list, tokenizer, model, max_length=150, num_beams=2):
    # We are going to add a prefix to each article to be summarized
    # so that the model knows what it should do
    prefix = "Summarize this news: "
    summaries_list = []  # Will contain all summaries

    texts_list = [prefix + text for text in texts_list]

    for text in texts_list:
        # Calculate the encodings
        input_encodings = tokenizer(
            text,
            max_length=1024,
            return_tensors='pt',
            padding=True,
            truncation=True
        )

        # Generate summaries
        with torch.no_grad():
            output = model.generate(
                input_ids=input_encodings.input_ids,
                attention_mask=input_encodings.attention_mask,
                max_length=max_length,  # Set the maximum length of the generated summary
                num_beams=num_beams,  # Set the number of beams for beam search
                early_stopping=True
            )

        # Decode to get the text
        summary = tokenizer.batch_decode(output, skip_special_tokens=True)
        summaries_list.append(summary)

    return summaries_list


In [11]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.4 MB/s eta 0:00:00


In [14]:
!pip install transformers

In [28]:
# Creating the summaries for both models.
summaries_small = create_summaries(articles,
 tokenizer_small,
 model_small)

summaries_reference = create_summaries(articles,
 tokenizer_reference,
 model_reference)

In [22]:
# Generate summaries
# with torch.no_grad():
#     output = model_small.generate(
#         input_ids=input_encodings,
#         max_length=150,  # Set the maximum length of the generated summary
#         num_beams=2,  # Set the number of beams for beam search
#         early_stopping=True
#     )

In [ ]:
# Rogue Evaluation

In [31]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=0030900ec5112adf4682e45770ab9cf7a76ec654ab1bb7f78fcf90dc1e32c3a3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [41]:
!pip install rouge


In [32]:
!pip install evaluate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00


In [33]:
import evaluate
from nltk.tokenize import sent_tokenize

#With the function load of the library evaluate
#we create a rouge_score object
rouge_score = evaluate.load("rouge")

In [ ]:
#create the compute_rouge_score function:

In [34]:
def compute_rouge_score(generated, reference):

 #We need to add '\n' to each line before send it to ROUGE
 generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
 reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]

 return rouge_score.compute(
 predictions=generated_with_newlines,
 references=reference_with_newlines,
 use_stemmer=True,

 )

In [ ]:
# making only a single call to the rouge_score.compute function

In [42]:
import nltk
nltk.download('punkt')
from rouge import Rouge
summaries_small = [str(summary) for summary in summaries_small]
summaries_reference = [str(summary) for summary in summaries_reference]

# Assuming summaries_small and summaries_reference are lists of strings
generated_with_newlines = ["\n".join(sent_tokenize(s)) for s in summaries_small]
reference_with_newlines = ["\n".join(sent_tokenize(s)) for s in summaries_reference]

# Compute ROUGE score
rouge = Rouge()
scores = rouge.get_scores(generated_with_newlines, reference_with_newlines, avg=True)

# Print ROUGE scores
print("ROUGE-1 F1 Score:", scores['rouge-1']['f'])
print("ROUGE-2 F1 Score:", scores['rouge-2']['f'])
print("ROUGE-L F1 Score:", scores['rouge-l']['f'])


ROUGE-1 F1 Score: 0.4479316068484054
ROUGE-2 F1 Score: 0.2852101191605464
ROUGE-L F1 Score: 0.43007446399126253


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
compute_rouge_score(summaries_small, summaries_reference)

{'rouge1': 0.47018752391886715,
 'rouge2': 0.3209013209013209,
 'rougeL': 0.34330271718331423,
 'rougeLsum': 0.44692881745120555}

In [44]:
#Comparing with a reference text

In [45]:
from datasets import load_dataset

cnn_dataset = load_dataset(
 "cnn_dailymail", version="3.0.0"
)

#Get just a few news to test
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS))

sample_cnn

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 3
})

In [46]:
#We obtain the maximum length of the selected summaries of the dataset,
#to indicate the maximum length that we want the response
# generated by the models to have.

In [47]:
max_length = max(len(item['highlights']) for item in sample_cnn)
max_length = max_length + 10

In [48]:
# We now have all the necessary to generate the summaries.

In [50]:
summaries_t5_base = create_summaries(sample_cnn["article"], tokenizer_small, model_small)
summaries_t5_finetuned = create_summaries(sample_cnn["article"], tokenizer_reference, model_reference)


#Get the real summaries from the cnn_dataset
real_summaries = sample_cnn['highlights']

In [51]:
#With the three summaries in their respective variables. Let’s look at the content

In [52]:
summaries = pd.DataFrame.from_dict(
 {
 "base": summaries_t5_base,
 "finetuned": summaries_t5_finetuned,
 "reference": real_summaries,
 }
 )
summaries.head()

,base,finetuned,reference
0,[a preliminary examination into alleged crimes...,[The Palestinian Authority becomes 123rd membe...,Membership gives the ICC jurisdiction over all...
1,[a stray pooch in Washington state has been bu...,[A stray bully breed mix has used up at least ...,"Theia, a bully breed mix, was apparently hit b..."
2,[mohammad Javad Zarif is the foreign minister ...,[Mohammad Javad Zarif has been Secretary of St...,Mohammad Javad Zarif has spent more time with ...


In [53]:
#The truth is that at a glance, it’s really difficult
#to know which of the two models creates better summaries.

#Let’s compute the ROUGE metrics for both models

In [66]:
##### Rogue score for _t5_finetuned

In [60]:
for summary in summaries_t5_finetuned:
    print(type(summary))


<class 'list'>
<class 'list'>
<class 'list'>


In [61]:
# Convert lists of lists into lists of strings
summaries_t5_finetuned = [" ".join(summary_list) for summary_list in summaries_t5_finetuned]
real_summaries = [" ".join(summary_list) for summary_list in real_summaries]


In [63]:
compute_rouge_score(summaries_t5_finetuned, real_summaries)

{'rouge1': 0.02830803354820822,
 'rouge2': 0.0,
 'rougeL': 0.02830803354820822,
 'rougeLsum': 0.02830803354820822}

In [65]:
########### Rogue score _t5_base

In [71]:
for summary in summaries_t5_base:
    print(type(summary))

<class 'list'>
<class 'list'>
<class 'list'>


In [74]:
# Convert lists to strings in summaries_t5_base
summaries_t5_base = [" ".join(summary) if isinstance(summary, list) else summary for summary in summaries_t5_base]

# Convert lists to strings in real_summaries
real_summaries = [" ".join(summary) if isinstance(summary, list) else summary for summary in real_summaries]

compute_rouge_score(summaries_t5_base, real_summaries)

{'rouge1': 0.05608143599775398,
 'rouge2': 0.0,
 'rougeL': 0.05608143599775398,
 'rougeLsum': 0.050020829937147925}